In [1]:
db_url = "https://github.com/cbdb-project/cbdb_sqlite/raw/master/latest.7z"
sql_url = "https://raw.githubusercontent.com/cbdb-project/biogref_CBDB/master/create_pd.sql"
db_filename = db_url.split("/")[-1]

In [2]:
import urllib.request
import os

if os.path.isfile(db_filename):
    os.remove(db_filename)

if os.path.isfile(db_filename.split(".")[0] + ".db"):
    os.remove(db_filename)

urllib.request.urlretrieve(db_url, db_filename)

('latest.7z', <http.client.HTTPMessage at 0x7f32976b3070>)

In [3]:
import requests

response = requests.get(sql_url)
sql_query = response.text

In [4]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(db_filename).extractall(".")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 1.8 MB/s eta 0:00:00


In [5]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(db_filename).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [6]:
print(sql_query)

SELECT BIOG_MAIN.c_personid as person_id, BIOG_MAIN.c_name_chn as person_name, 
CASE
    WHEN BIOG_MAIN.c_female= 0 THEN '男'
    WHEN BIOG_MAIN.c_female<> 0 THEN '女'
END as gender,
BIOG_MAIN.c_birthyear as born_year, BIOG_MAIN.c_deathyear as died_year, DYNASTIES.c_dynasty_chn as dynasty, ADDR_CODES.c_name_chn as jiguan
FROM BIOG_MAIN LEFT JOIN ADDR_CODES ON BIOG_MAIN.c_index_addr_id = ADDR_CODES.c_addr_id LEFT JOIN DYNASTIES ON DYNASTIES.c_dy = BIOG_MAIN.c_dy
WHERE c_personid != 0


In [7]:
data_pd = pd.read_sql_query(sql_query, CONN).drop_duplicates()
data_pd['born_year'] = data_pd['born_year'].fillna(9999).astype(int)
data_pd['died_year'] = data_pd['died_year'].fillna(9999).astype(int)
data_pd = data_pd.replace({'\",\n': ''}, regex=True).replace({'.*未詳.*': ''}, regex=True).replace(9999,'')
data_pd.fillna('')

,person_id,person_name,gender,born_year,died_year,dynasty,jiguan
0,1,安惇,男,1042,1104,宋,渠江
1,2,安邡,男,,,宋,渠江
2,3,安燾,男,,,宋,開封
3,4,查道,男,955,1018,宋,休寧
4,5,查籥,男,,,宋,江陵
...,...,...,...,...,...,...,...
529554,570007,沈氏(金璐妻),女,,,清,山陰
529555,570008,陸氏(張遵訓妻),女,,,清,山陰
529556,570009,戴學連,男,,,清,山陰
529557,570010,潘恆貞,女,,,清,


In [11]:
data_pd.to_csv('data.csv', index=True, index_label='primary_id')
if os.path.isfile(db_filename):
    os.remove(db_filename)

if os.path.isfile(db_filename.split(".")[0] + ".db"):
    os.remove(db_filename.split(".")[0] + ".db")